In [1]:
import taxcalc as tc
from pathlib import Path
import os
import numpy as np

In [2]:
# Policy1:  
# Base EITC on individual earnings, EITC only available for tax units with <$100,000

In [22]:
recs = tc.Records.cps_constructor()
pol = tc.Policy()
calc0 = tc.Calculator(policy=pol, records=recs)

In [23]:
# reform2: combine effect
reform2 = {"EITC_indiv": {"2022": True}}
pol2 = tc.Policy()
pol2.implement_reform(reform2, print_warnings=False, raise_errors=False)
calc2 = tc.Calculator(policy=pol2, records=recs)

In [24]:
totalam = 0
# Policy1: switch EITC from household to individuals & celling of $100,000 income on tax units
for year in range(2023, 2034):   
    print("year: ", year)
    calc0.advance_to_year(year)
    calc0.calc_all()
    itax_rev0 = calc0.weighted_total('iitax')
    print("current law income tax revenue: ", itax_rev0)
    calc2.advance_to_year(year)
    calc2.calc_all()
    
    # calculate the difference of total EITC between the limited (household with income lower than $100,000) and normal, for reform2
    vardf = calc2.dataframe(['s006', 'c00100', 'eitc', 'MARS'])
    lvardf = vardf.loc[vardf['c00100'] < 100000]
    
    vardf = tc.add_income_table_row_variable(vardf, 'c00100', tc.SOI_AGI_BINS)
    gbydf = vardf.groupby('table_row', as_index=False)
    
    lvardf = tc.add_income_table_row_variable(lvardf, 'c00100', tc.SOI_AGI_BINS)
    lgbydf = lvardf.groupby('table_row', as_index=False)

    tot_amount = 0.
    for grp_interval, grp in gbydf:
        amount = (grp['eitc'] * grp['s006']).sum()
        tot_amount += amount

    ltot_amount = 0.
    for lgrp_interval, lgrp in lgbydf:
        lamount = (lgrp['eitc'] * lgrp['s006']).sum() 
        ltot_amount += lamount

    eitc_diff = ltot_amount - tot_amount
    itax_rev2 = calc2.weighted_total('iitax')
    

    # CHANGE 
    litax = itax_rev2 + tot_amount - ltot_amount 
    # litax = itax_rev2 + eitc_diff
    newcost = litax - itax_rev0
    totalam = totalam + newcost
    print("reformed policy with indiv switch & income celling ~ income tax revenue: ", litax)
    print("difference of income revenue: reform vs current law ", litax - itax_rev0 )



year:  2023
current law income tax revenue:  1714313379269.0566
reformed policy with indiv switch & income celling ~ income tax revenue:  1671662512025.5447
difference of income revenue: reform vs current law  -42650867243.51196
year:  2024
current law income tax revenue:  1782172849003.1306
reformed policy with indiv switch & income celling ~ income tax revenue:  1739031274618.5742
difference of income revenue: reform vs current law  -43141574384.5564
year:  2025
current law income tax revenue:  1878751353093.122
reformed policy with indiv switch & income celling ~ income tax revenue:  1836296048786.1255
difference of income revenue: reform vs current law  -42455304306.99658
year:  2026
current law income tax revenue:  2198679845513.8442
reformed policy with indiv switch & income celling ~ income tax revenue:  2157133485625.8152
difference of income revenue: reform vs current law  -41546359888.02905
year:  2027
current law income tax revenue:  2305369604977.295
reformed policy with in

In [6]:
# Policy2: 
# Base EITC on individual earnings, EITC only available for TUs <$100K, 
# eliminate EITC for single childless adults (BUT NOT FOR MARRIED CHILDLESS ADULTS)

In [7]:
# define the calculator object with data file and policy
recs = tc.Records.cps_constructor()
pol = tc.Policy()
calc0 = tc.Calculator(policy=pol, records=recs)

In [8]:
# reform2: swich to indiv
reform2 = {"EITC_indiv": {"2022": True}}
pol2 = tc.Policy()
pol2.implement_reform(reform2, print_warnings=False, raise_errors=False)
calc2 = tc.Calculator(policy=pol2, records=recs)


In [9]:
# reform4: only calculate the single EITC
reform4 = {"EITC_indiv": {"2022": True},
           "EITC_c": {"2022": [0, 3733.0, 6164.0, 6935.0]}}
             
pol4 = tc.Policy()
pol4.implement_reform(reform4, print_warnings=False, raise_errors=False)
calc4 = tc.Calculator(policy=pol4, records=recs)

In [10]:
totalam = 0
for year in range(2023, 2034):   
    print("year: ", year)
    calc0.advance_to_year(year)
    calc0.calc_all()
    
    EITC_rev0 = calc0.weighted_total('eitc')
    itax_rev0 = calc0.weighted_total('iitax')
    print("current law income tax revenue: ", itax_rev0)

    
    # calculate single only eitc amount
    calc4.advance_to_year(year)
    calc4.calc_all()
    fvardf = calc4.dataframe(['s006', 'c00100', 'eitc', 'MARS'])
    # select single only    &  select income lower than 100000
    flvardf = fvardf.loc[fvardf['MARS'] == 1].loc[fvardf['c00100'] < 100000] 
    flvardf = tc.add_income_table_row_variable(flvardf, 'c00100', tc.SOI_AGI_BINS)
    flgbydf = flvardf.groupby('table_row', as_index=False)
    
    fltot_amount = 0.
    for lgrp_interval, lgrp in flgbydf:
        lamount = (lgrp['eitc'] * lgrp['s006']).sum() 
        fltot_amount += lamount
    

    # calculate non-single eitc
    calc2.advance_to_year(year)
    calc2.calc_all()
    vardf = calc2.dataframe(['s006', 'c00100', 'eitc', 'MARS'])
    lvardf = vardf.loc[vardf['MARS'] != 1].loc[vardf['c00100'] < 100000]
    lvardf = tc.add_income_table_row_variable(lvardf, 'c00100', tc.SOI_AGI_BINS)
    lgbydf = lvardf.groupby('table_row', as_index=False)

    ltot_amount = 0.
    for lgrp_interval, lgrp in lgbydf:
        lamount = (lgrp['eitc'] * lgrp['s006']).sum() 
        ltot_amount += lamount


    # total eitc married + non-married
    totaleitc = ltot_amount + fltot_amount
    
    # income tax revenue
    reformed_iitax = calc4.weighted_total('iitax') + calc4.weighted_total('eitc') - totaleitc 

    newcost = totaleitc - EITC_rev0
    totalam = totalam + newcost
    # print("reform 4 single eitc ", fltot_amount )
    # print("reform 2 non single eitc: ", ltot_amount)
    # print("total eitc: ", totaleitc)
    
    print("reform: ", reformed_iitax)
    print("difference of income revenue: reform vs current law", (calc4.weighted_total('iitax') + calc4.weighted_total('eitc') - totaleitc) - calc0.weighted_total('iitax')) 
    # (calc0.weighted_total('iitax') + calc0.weighted_total('eitc') +  calc0.weighted_total('c11070')) always equal
    print("difference of eitc: reform vs current law ", totaleitc - EITC_rev0)
    
#print("note: the income revenue will decrease, the total eitc will increase; these two values are basically same but with little difference casued by CTC")
    


year:  2023
current law income tax revenue:  1714313379269.0566
reform:  1674026823420.695
difference of income revenue: reform vs current law -40286555848.36157
difference of eitc: reform vs current law  40285168207.11665
year:  2024
current law income tax revenue:  1782172849003.1306
reform:  1741528335109.2866
difference of income revenue: reform vs current law -40644513893.843994
difference of eitc: reform vs current law  40643032320.323166
year:  2025
current law income tax revenue:  1878751353093.122
reform:  1838830147136.6914
difference of income revenue: reform vs current law -39921205956.430664
difference of eitc: reform vs current law  39919683772.18675
year:  2026
current law income tax revenue:  2198679845513.8442
reform:  2159665219381.8074
difference of income revenue: reform vs current law -39014626132.036865
difference of eitc: reform vs current law  39012627611.264786
year:  2027
current law income tax revenue:  2305369604977.295
reform:  2267253764594.364
difference 

In [11]:
# Policy3
# Base EITC on individual earnings, EITC only available for TUs <$100K, 
# replace the married parent EITC schedules with the “super-schedule” for married couples from Winship 2021 paper

In [12]:
# define the calculator object with data file and policy
recs = tc.Records.cps_constructor()
pol = tc.Policy()
calc0 = tc.Calculator(policy=pol, records=recs)

In [13]:
# reform2: combine effect
reform2 = {"EITC_indiv": {"2022": True}}
pol2 = tc.Policy()
pol2.implement_reform(reform2, print_warnings=False, raise_errors=False)
calc2 = tc.Calculator(policy=pol2, records=recs)

In [14]:
# reform4: swich + super schedule -- only calculate the married EITC
reform4 = {"EITC_indiv": {"2022": True},
           "EITC_c": {"2022": [0, 6935.0, 6935.0, 6935.0]},
           "EITC_rt": {"2022": [0.45, 0.45, 0.45, 0.45]},
           "EITC_prt": {"2022": [0.24, 0.24, 0.24, 0.24]},
           "EITC_ps": {"2022": [20130.0, 20130.0, 20130.0, 20130.0]},
           "EITC_ps_MarriedJ": {"2022": [18390.0, 18390.0, 18390.0, 18390.0]}
             }
pol4 = tc.Policy()
pol4.implement_reform(reform4, print_warnings=False, raise_errors=False)
calc4 = tc.Calculator(policy=pol4, records=recs)

In [15]:
totalam = 0
for year in range(2023, 2034):   
    print("year: ", year)
    calc0.advance_to_year(year)
    calc0.calc_all()
    
    EITC_rev0 = calc0.weighted_total('eitc')
    itax_rev0 = calc0.weighted_total('iitax')
    print("current law income tax revenue: ", itax_rev0)

    
    # calculate married only eitc amount
    calc4.advance_to_year(year)
    calc4.calc_all()
    fvardf = calc4.dataframe(['s006', 'c00100', 'eitc', 'MARS'])
    # select married only    &  select income lower than 100000
    flvardf = fvardf.loc[fvardf['MARS'] == 2].loc[fvardf['c00100'] < 100000] 
    flvardf = tc.add_income_table_row_variable(flvardf, 'c00100', tc.SOI_AGI_BINS)
    flgbydf = flvardf.groupby('table_row', as_index=False)
    
    fltot_amount = 0.
    for lgrp_interval, lgrp in flgbydf:
        lamount = (lgrp['eitc'] * lgrp['s006']).sum() 
        fltot_amount += lamount
    

    # calculate single eitc
    calc2.advance_to_year(year)
    calc2.calc_all()
    vardf = calc2.dataframe(['s006', 'c00100', 'eitc', 'MARS'])
    lvardf = vardf.loc[vardf['MARS'] != 2].loc[vardf['c00100'] < 100000]
    lvardf = tc.add_income_table_row_variable(lvardf, 'c00100', tc.SOI_AGI_BINS)
    lgbydf = lvardf.groupby('table_row', as_index=False)

    ltot_amount = 0.
    for lgrp_interval, lgrp in lgbydf:
        lamount = (lgrp['eitc'] * lgrp['s006']).sum() 
        ltot_amount += lamount
        

    # total eitc married + non-married
    totaleitc = ltot_amount + fltot_amount
    
    # income tax revenue
    reformed_iitax = calc4.weighted_total('iitax') + calc4.weighted_total('eitc') - totaleitc 

    newcost = totaleitc - EITC_rev0
    totalam = totalam + newcost
    # print("reform 4 married eitc ", fltot_amount )
    # print("reform 2 non married eitc: ", ltot_amount)
    # print("total eitc: ", totaleitc)
    
    print("reform: ", reformed_iitax)
    print("difference of income revenue: reform vs current law", (calc4.weighted_total('iitax') + calc4.weighted_total('eitc') - totaleitc) - calc0.weighted_total('iitax')) 
    # (calc0.weighted_total('iitax') + calc0.weighted_total('eitc') +  calc0.weighted_total('c11070')) always equal
    print("difference of eitc: reform vs current law ", totaleitc - EITC_rev0)
    
print("note: the income revenue will decrease, the total eitc will increase; these two values are basically same but with little difference casued by CTC")
    

year:  2023
current law income tax revenue:  1714313379269.0566
reform:  1632029862155.4438
difference of income revenue: reform vs current law -82283517113.6128
difference of eitc: reform vs current law  82282129472.36752
year:  2024
current law income tax revenue:  1782172849003.1306
reform:  1699148332360.8801
difference of income revenue: reform vs current law -83024516642.25049
difference of eitc: reform vs current law  83023035068.729
year:  2025
current law income tax revenue:  1878751353093.122
reform:  1797072876747.504
difference of income revenue: reform vs current law -81678476345.61816
difference of eitc: reform vs current law  81676954161.37439
year:  2026
current law income tax revenue:  2198679845513.8442
reform:  2118583083186.526
difference of income revenue: reform vs current law -80096762327.31836
difference of eitc: reform vs current law  80094763806.54634
year:  2027
current law income tax revenue:  2305369604977.295
reform:  2226797175540.6978
difference of incom

In [16]:
# Policy4
# Base EITC on individual earnings, EITC only available for TUs <$100K, 
# eliminate EITC for single childless adults (BUT NOT FOR MARRIED CHILDLESS ADULTS),
# replace the married parent EITC schedules with the “super-schedule” for married couples from Winship 2021 paper

In [17]:
# define the calculator object with data file and policy
recs = tc.Records.cps_constructor()
pol = tc.Policy()
calc0 = tc.Calculator(policy=pol, records=recs)

In [18]:
# reform1: switch only
reform1 = {"EITC_indiv": {"2022": True}}
pol1 = tc.Policy()
pol1.implement_reform(reform1, print_warnings=False, raise_errors=False)
calc1 = tc.Calculator(policy=pol1, records=recs)

In [19]:
# reform2: only calculate the single EITC
reform2 = {"EITC_indiv": {"2022": True},
           "EITC_c": {"2022": [0, 3733.0, 6164.0, 6935.0]}}
             
pol2 = tc.Policy()
pol2.implement_reform(reform2, print_warnings=False, raise_errors=False)
calc2 = tc.Calculator(policy=pol2, records=recs)

In [20]:
# reform4: swich + super schedule -- only calculate the married EITC
reform4 = {"EITC_indiv": {"2022": True},
           "EITC_c": {"2022": [0, 6935.0, 6935.0, 6935.0]},
           "EITC_rt": {"2022": [0.45, 0.45, 0.45, 0.45]},
           "EITC_prt": {"2022": [0.24, 0.24, 0.24, 0.24]},
           "EITC_ps": {"2022": [20130.0, 20130.0, 20130.0, 20130.0]},
           "EITC_ps_MarriedJ": {"2022": [18390.0, 18390.0, 18390.0, 18390.0]}
             }
pol4 = tc.Policy()
pol4.implement_reform(reform4, print_warnings=False, raise_errors=False)
calc4 = tc.Calculator(policy=pol4, records=recs)

In [21]:
totalam = 0

for year in range(2023, 2034):   
    print("year: ", year)
    calc0.advance_to_year(year)
    calc0.calc_all()
    
    EITC_rev0 = calc0.weighted_total('eitc')
    itax_rev0 = calc0.weighted_total('iitax')
    print("current law income tax revenue: ", itax_rev0)

    
    # calculate married only eitc amount -- super schedule
    calc4.advance_to_year(year)
    calc4.calc_all()
    fvardf = calc4.dataframe(['s006', 'c00100', 'eitc', 'MARS'])
    # select married only    &  select income lower than 100000
    flvardf = fvardf.loc[fvardf['MARS'] == 2].loc[fvardf['c00100'] < 100000] 
    flvardf = tc.add_income_table_row_variable(flvardf, 'c00100', tc.SOI_AGI_BINS)
    flgbydf = flvardf.groupby('table_row', as_index=False)
    
    fltot_amount = 0.
    for lgrp_interval, lgrp in flgbydf:
        lamount = (lgrp['eitc'] * lgrp['s006']).sum() 
        fltot_amount += lamount
    

    # calculate single eitc -- zero out childless
    calc2.advance_to_year(year)
    calc2.calc_all()
    vardf = calc2.dataframe(['s006', 'c00100', 'eitc', 'MARS'])
    lvardf = vardf.loc[vardf['MARS'] == 1].loc[vardf['c00100'] < 100000]
    lvardf = tc.add_income_table_row_variable(lvardf, 'c00100', tc.SOI_AGI_BINS)
    lgbydf = lvardf.groupby('table_row', as_index=False)

    ltot_amount = 0.
    for lgrp_interval, lgrp in lgbydf:
        lamount = (lgrp['eitc'] * lgrp['s006']).sum() 
        ltot_amount += lamount
        

    calc1.advance_to_year(year)
    calc1.calc_all()
    nvardf = calc1.dataframe(['s006', 'c00100', 'eitc', 'MARS'])
    # mars is 3 or 4 or 5
    nlvardf = nvardf.loc[(nvardf['MARS'] == 3) | (nvardf['MARS'] == 4) | (nvardf['MARS'] == 5)].loc[nvardf['c00100'] < 100000]
    nlvardf = tc.add_income_table_row_variable(nlvardf, 'c00100', tc.SOI_AGI_BINS)
    nlgbydf = nlvardf.groupby('table_row', as_index=False)

    nltot_amount = 0.
    for lgrp_interval, lgrp in nlgbydf:
        lamount = (lgrp['eitc'] * lgrp['s006']).sum() 
        nltot_amount += lamount    
        
        
        
        
        
    # total eitc married + non-married
    totaleitc = ltot_amount + fltot_amount + nltot_amount
    
    # income tax revenue
    reformed_iitax = calc4.weighted_total('iitax') + calc4.weighted_total('eitc') - totaleitc 


    # print("reform 4 married eitc ", fltot_amount )
    # print("reform 2 non married eitc: ", ltot_amount)
    # print("total eitc: ", totaleitc)
    newcost = totaleitc - EITC_rev0
    totalam = totalam + newcost
    
    print("reform: ", reformed_iitax)
    print("difference of income revenue: reform vs current law", (calc4.weighted_total('iitax') + calc4.weighted_total('eitc') - totaleitc) - calc0.weighted_total('iitax')) 
    # (calc0.weighted_total('iitax') + calc0.weighted_total('eitc') +  calc0.weighted_total('c11070')) always equal
    print("difference of eitc: reform vs current law ", totaleitc - EITC_rev0)
    
print("note: the income revenue will decrease, the total eitc will increase; these two values are basically same but with little difference casued by CTC")
    

year:  2023
current law income tax revenue:  1714313379269.0566
reform:  1634394822588.9688
difference of income revenue: reform vs current law -79918556680.08789
difference of eitc: reform vs current law  79917169038.8424
year:  2024
current law income tax revenue:  1782172849003.1306
reform:  1701645973898.9268
difference of income revenue: reform vs current law -80526875104.20386
difference of eitc: reform vs current law  80525393530.68219
year:  2025
current law income tax revenue:  1878751353093.122
reform:  1799607480366.688
difference of income revenue: reform vs current law -79143872726.43408
difference of eitc: reform vs current law  79142350542.19034
year:  2026
current law income tax revenue:  2198679845513.8442
reform:  2121115767297.4375
difference of income revenue: reform vs current law -77564078216.40674
difference of eitc: reform vs current law  77562079695.63478
year:  2027
current law income tax revenue:  2305369604977.295
reform:  2229334809159.2637
difference of in